In [ ]:
%%bash
lein uberjar

In [ ]:
%classpath add jar ../target/jvm/uberjar/podsm-0.1.0-SNAPSHOT-standalone.jar
(clojure.lang.Compiler/loadFile "jupyter_helpers.clj")
(require '[clojure.java.io :as io]
         '[clojure.string :as string]
         '[clojure.pprint :refer [pprint print-table]]
         '[jupyter-helpers :refer [run-and-save-experiments load-experiment site-summary-table experiment-timeline experiments-summary-table
                                   display-html confusion-summary display-experiment-models get-best-experiment-label load-experiment-pair
                                   get-timing-evaluation timing-table get-experiment-block-accuracies experiment-block-accuracies-table
                                   save-data load-data]]
         '[podsm.evaluation :refer [get-order-summary]]
         '[podsm.classifier.distributed.sites :refer [make-site-structure p-site t-site]]
         '[podsm.classifier.moa-classifier :refer [adaptive-random-forest naive-bayes]]
         '[podsm.dataset.csv-dataset :refer [read-csv-dataset]]
         '[podsm.utils.stats :refer [mann-whitney-u-test wilcoxon-signed-rank-test]])

## Experiment Setup

In [ ]:
(defn dataset-fn []
    (read-csv-dataset "datasets/asthma/asthma-resampled.csv"
                      999999 ;; All records
                      ;; Numeric features (all except class)
                      ["capped_ozone","capped_bc(370)","capped_bc(880)","avg_no_x","UVI","avg PM","avg_pm","avg_temp", "capped_wind",
                      "capped_Tair", "capped_Twet","capped_Tdew","capped_RH", "Prec","capped_pres","avg_no_y","capped_scale"]))

(def dataset-name "asthma")
(def feature-count 17) ;; Not including class
(def features-per-site 1)

(def trouble-factors (map #(* features-per-site %) [1 1.5 2]))
(def p-sites
    [(p-site :p-site1 [0])
     (p-site :p-site2 [1 2])
    ;; (p-site :p-site3 [2])
     (p-site :p-site3 [3])
     (p-site :p-site4 [4])
     (p-site :p-site5 [5])
     (p-site :p-site6 [6])
     (p-site :p-site7 [7 14])
     (p-site :p-site8 [8])
     (p-site :p-site9 [9 10 11 12])
    ;;  (p-site :p-site11 [10])
     ;;  (p-site :p-site12 [11])
      ;;  (p-site :p-site13 [12])
         (p-site :p-site10 [13])
      ;;   (p-site :p-site15 [14])
         (p-site :p-site11 [15])
         (p-site :p-site12 [16])
         
         ])


(def dataset (dataset-fn))
(defn dataset-fn [] dataset)


In [ ]:
;; Experiment Configuration

(def base-classifier naive-bayes)
(def trouble-classifier adaptive-random-forest)

(def base-setup
    {:dataset-description (keyword dataset-name)
     :dataset-fn dataset-fn
     :trouble-classifier trouble-classifier
     :base-site-structure (apply make-site-structure feature-count p-sites)
     ;; Disable monitor logging because this is a long-running experiment
     ;; where detailed logging data is excessively large.
     :disable-monitor-logging true
     :batch1 15000
     })

(def aggregation-rules {:max-conf {:type :max-conf}
                       :simple-voting {:type :simple-voting}
                        :stacked {:type :two-level-stacked
                         :classifier-generator trouble-classifier}
    })

(def base-system-config
    {:site-window-size 1000
     :site-training-time 0
     :shared-sources? false
     :creation-window-size 1000
     :creation-time-threshold 500
     :removal-window-size 1000
     :removal-time-threshold 500
     :creation-agreement-threshold {:type :smoothed-hoeffding-bound
                                    :r 1 :delta 0.001 :sharpness 5}
     :removal-accuracy-threshold {:type :hoeffding-bound
                                  :r 1 :delta 0.001}
     :removal-usage-threshold 0.05
 
     })

(def naive-experiments
    (for [[aggregation-rule-key aggregation-rule] aggregation-rules]
        (assoc base-setup
            :label (keyword (str dataset-name "-naive" aggregation-rule-key))
            :p-site-aggregation-rule aggregation-rule
            :system-config :naive)))

(def park-experiments
    (for [trouble-factor trouble-factors]
        (assoc base-setup
            :label (keyword (str dataset-name "-park"
                                 "-tf" trouble-factor))
            :base-site-structure (apply make-site-structure feature-count
                                     (concat p-sites [(t-site :trouble (map :label p-sites) trouble-factor)]))
            :disable-monitors true
            :p-site-aggregation-rule {:type :max-conf}
            :system-config (assoc base-system-config
                               :trouble-factor trouble-factor
                               :creation-agreement-threshold 999
                               :removal-accuracy-threshold -999
                               :removal-usage-threshold -999))))
(def podsm-experiments
    (for [trouble-factor trouble-factors
          [aggregation-rule-key aggregation-rule] aggregation-rules]
        (assoc base-setup
            :label (keyword (str dataset-name "-podsm"
                                 aggregation-rule-key
                                 "-tf" trouble-factor))
            :p-site-aggregation-rule aggregation-rule
            :system-config (assoc base-system-config
                               :trouble-factor trouble-factor
                               :batch1 15000
                               ))))

(def experiments  podsm-experiments) 

(def output-dir (str "workspace/" dataset-name))

## Run Experiments

In [ ]:
(import '[com.twosigma.beakerx.widget Output])
(def experiment-output (Output.))
experiment-output

In [ ]:
(run-and-save-experiments output-dir base-classifier experiments
                          :beaker-output experiment-output
                          :thread-count 1)

## Experiments Summary

In [ ]:
(def experiments-summary
    (load-experiment output-dir :summary))

(experiments-summary-table experiments-summary)

## Accuracy and Transmission Over Time

In [ ]:
(def performance-measure [:accuracy])
(def partition-size 100)

### Maximum Confidence Aggregation

In [ ]:
(def max-conf-experiments (load-experiment-pair output-dir experiments-summary performance-measure :max-conf))

In [ ]:
(let [experiments max-conf-experiments
      result-count (count (:results (:podsm experiments)))]
    (.display (experiment-timeline (:podsm experiments)
                                   (dissoc experiments :podsm)
                                   :partition-size partition-size
                                   :x-bounds [0 result-count]
                                   :y-bounds-acc [0 1.2]
                                   :y-bounds-data [0 1.5]
                                    :y-bounds-timing [1 10]
                                   :include-timing-plot? true
                                    ;;:y-bounds-agree [0 300]
                                   :event-rows 6)))

### Simple Voting Aggregation

In [ ]:
(def voting-experiments (load-experiment-pair output-dir experiments-summary performance-measure :simple-voting))

In [ ]:
(let [experiments voting-experiments
      result-count (count (:results (:podsm experiments)))]
    (.display (experiment-timeline (:podsm experiments)
                                   (dissoc experiments :podsm)
                                   :partition-size partition-size
                                   :x-bounds [0 result-count]
                                  :y-bounds-acc [0 1.2]
                                   :y-bounds-data [0 1.5]
                                       :y-bounds-timing [1 10]
                                   :include-timing-plot? true
                                   ;; :y-bounds-agree [0 300]
                                   :event-rows 6)))

### Stacked Aggregation

In [ ]:
(def stacked-experiments (load-experiment-pair output-dir experiments-summary performance-measure :stacked))

In [ ]:
(let [experiments stacked-experiments
      result-count (count (:results (:podsm experiments)))]
    (.display (experiment-timeline (:podsm experiments)
                                   (dissoc experiments :podsm)
                                   :partition-size partition-size
                                   :x-bounds [0 result-count]
                                   :y-bounds-acc [0 1.2]
                                   :y-bounds-data [0 1.5]
                                       :y-bounds-timing [1 10]
                                   :include-timing-plot? true
                                  ;;  :y-bounds-agree [0 300]
                                   :event-rows 6)))

## Timing Evaluation

In [ ]:
(def timing-evaluation (get-timing-evaluation output-dir
                                              (distinct (map :label experiments-summary)) ;; Drop repeated experiment labels
                                              1000))

In [ ]:
(save-data (str output-dir "/:timing-summary.edn") timing-evaluation)

In [ ]:
(timing-table timing-evaluation)

## Centralized Configuration

In [ ]:
(def central-output-dir (str output-dir "/central"))
(def centralise-all-trouble-factor (* 2 feature-count))
(def centralised-experiment
    (assoc base-setup
        :label (keyword (str dataset-name "-centralised"))
        :base-site-structure
        (make-site-structure feature-count (p-site :central (range feature-count)))
       ;;(apply make-site-structure feature-count
              ;; (concat p-sites [(t-site :central (map :label p-sites) centralise-all-trouble-factor)]))
        :p-site-aggregation-rule {:type :max-conf}
         :disable-monitors true
        :system-config (assoc base-system-config,
                                :trouble-factor centralise-all-trouble-factor,
                                :creation-agreement-threshold 999,
                                :removal-accuracy-threshold -999,
                                :removal-usage-threshold -999))),
    
        
(run-and-save-experiments central-output-dir trouble-classifier [centralised-experiment]
                          :thread-count 1)

In [ ]:
(def central-experiments-summary
    (load-experiment central-output-dir :summary))
    
(experiments-summary-table central-experiments-summary)

In [ ]:
(def timing-evaluation (get-timing-evaluation (str output-dir "/central")
                                              (distinct (map :label central-experiments-summary)) ;; Drop repeated experiment labels
                                              1000))

In [ ]:
(save-data (str output-dir "/:timing-summary.edn") timing-evaluation)

In [ ]:
(timing-table timing-evaluation)